In [663]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *

# Insect Wing Deformation

**Background:**

Wing is modeled as a cantilever beam with distributed load and a joint where the material property changes.

**Purpose:**  

The purpose of this program is to model an insect's wing that has minimized stress while optimizing the material properties to allow for some deformation.

**Assumptions made:**
   * uniform mass distribution
   * isotropic material 
   * inertial forces are roughly equal in magnitude to aeordynamic forces (air load)
  
    


Constants:

In [664]:
wingLength = 0.09 # units = m
jointStartLocation = 0.04
jointSize = 0.001
wingRadius = 0.001 # units = m
angleOfWing = 0.2
load = -240
young = 610000000  # E for chitin 
j_young = 2000000 #units = N/m^2 / E for resilin
air_load = -14.43 # 
wingWeight = 240 #units = ug
minimumStress = 1
thresholdStress = 3
maxDeflection = 0.01

In [665]:
def wingSim(wingWeight, young, j_young, air_load, wingLength, jointStartLocation, jointSize, wingRadius, angleOfWing, minimumStress, thresholdStress):
    daMoment = getSecondMoment(wingRadius)
    wDistrib = wingWeight/ wingLength
    totalD = wDistrib + air_load
    #incorporating the angle change
    trueTotalD = totalD * cos(angleOfWing)
    themSupports = getSupportReactions(trueTotalD, wingLength)
    listODeflections = getDeflection(young, j_young, trueTotalD, wingLength, jointStartLocation, jointSize, daMoment)
    x, y = symbols('x'), symbols('y')
    momentExpr = -trueTotalD * ((wingLength - x)**2)*(1/2)
    maxMoment = momentExpr.subs(x, 0) 
#     print(maxMoment)
    #getting best radiuses for the given wingLength
    rangeOfRadii = getOptimalradius(maxMoment, minimumStress, thresholdStress, wingLength)
    print("Minimum radius: ", rangeOfRadii[0])
    print("Maximum radius: ", rangeOfRadii[1])

    #getting best wing lengths for given radius
    w = symbols('w')
    maxMoment2 = -trueTotalD * ((w) ** 2) * 1/2
    
    rangeOfWings = getOptimalWingLength(maxMoment2, minimumStress, thresholdStress, wingRadius)
    
    
    #getYoung's Moduli
    getBestMaterials(rangeOfRadii, rangeOfWings, jointStartLocation, jointSize, totalD, maxDeflection, young)
    
    #choosing the positive root
    print("Minimum wing length: ", rangeOfWings[0])
    print("Maximum wing length: ", rangeOfWings[1])

In [666]:
def getBestMaterials(rangeOradii, rangeOwings, jointStart, jointSize, load, maxDeflection, young):
    minLength = rangeOwings[0]
    maxLength = rangeOwings[1]
    minRadius = rangeOradii[0]
    maxRadius = rangeOradii[1]
    j, x = symbols('j'), symbols('x')
    #for minimum stress side of deflection with the shortest wing and largest radius
    minMoment = getSecondMoment(maxRadius)
    minDeflections = getDeflection(j, j_young, load, minLength, jointStart, jointSize, minMoment)
#     print(minDeflections[2])
    deflectionAtEnd =  minDeflections[2].subs(x, minLength)
#     print(deflectionAtEnd)
    
    minimumYoung1 = solve(deflectionAtEnd - maxDeflection, j)[0]
    print("Minimum Young's Modulus: ", minimumYoung1)
#     print("Min Bending stiffness (at the tip): ", minimumYoung1*minMoment)

    maxMoment = getSecondMoment(minRadius)
    maxDeflections = getDeflection(j, j_young, load, maxLength, jointStart, jointSize, maxMoment)

    
    deflectionAtEnd =  maxDeflections[2].subs(x, maxLength)
    minimumYoung2 = solve(deflectionAtEnd - maxDeflection, j)[0]
    print("Maximum Young's Modulus: ", minimumYoung2)
    print("Bending stiffness (at the tip, EI(L)): ", minimumYoung2*maxMoment)
    
    
    deflectionAtBase =  minDeflections[2].subs(x, 0)
    otherYoung1 = solve(deflectionAtBase, j)[0]
    print("Bending stiffness (at the wing base, EI(0)-->zero deflection): ", otherYoung1*maxMoment)
    

In [667]:
def getSupportReactions(totalD, wingLength):
    x, y = symbols('x'), symbols('y')
    #since there are no forces in the x-direction, the support in the d direction would be 0
    xSup = 0
    ySup = integrate(totalD, (x, 0, wingLength)) 

    #we can integrate the total distribution over the length of the wing to find the moment
    mSup = integrate(totalD * x, (x, 0, wingLength))
    return [xSup, ySup, mSup]

In [668]:
def getOptimalWingLength(maxMoment, minimumStress, threshold, radius):
    w = symbols ('w')

    ratio = radius / getSecondMoment(radius)
    
    possibleStresses = -(ratio * maxMoment)
    upperBound = solve(possibleStresses - threshold, w)
    lowerBound = solve(possibleStresses - minimumStress, w)


    return [lowerBound[1], upperBound[1]]

In [669]:
def getOptimalradius(maxMoment1, minimumStress, threshold, wingLength):
    #maximum stress is at the tip of the cross section
    
    r = symbols('r')
    moment = (pi/4) * r **4
    ratioExpr = r/moment
    #print(ratioExpr)
    #maximum moment is at the tip of the wing since the derivative of the moemnt 
    #is linear with a y-intercept at the orign. 

    #ratio of the radius of the wing to its moment.
    
    
    #relationship between stress and moment
    possibleMaxStresses = -(ratioExpr * maxMoment1)
    #print(possibleMaxStresses)
    
    #getting lower and upper bounds by seeing what the optimum radius is when possibleMaxStress= minimumStress or = threshold
    upperBound = solve((possibleMaxStresses - minimumStress), r)
    lowerBound = solve((possibleMaxStresses - threshold), r)
    #print(lowerBound)
    #print(upperBound)
    return [lowerBound[0], upperBound[0]]


In [670]:
def getSupportReactions(totalD, wingLength):
    x, y = symbols('x'), symbols('y')
    #since there are no forces in the x-direction, the support in the d direction would be 0
    xSup = 0
    ySup = integrate(totalD, (x, 0, wingLength)) 

    #we can integrate the total distribution over the length of the wing to find the moment
    mSup = integrate(totalD * x, (x, 0, wingLength))
    return [xSup, ySup, mSup]

In [671]:
def getDeflection(young, j_young, trueTotalD, wingLength, jointStartLocation, jointSize, daMoment):
    
    x, y = symbols('x'), symbols('y')
    #Calculating Moment. Suppose cut at x. Only one cut necessary since the moment doesn't depend on material properties.
    momentExpr = -trueTotalD * ((wingLength - x)**2)*(1/2)
    #print(momentExpr)

    #between x = 0 to x = jointStartLocation (assuming JointStartLocation < wingLength)
    deflectionExpr1 = calculateDeflection(young, momentExpr, daMoment)
    #both constants of integration = 0 since at x = 0, v(x) = 0 and dv/dx = 0

    #at the joint (between jointStartLocation and (jointStartLocation + jointLength))
    draftdeflectionExpr2 = calculateDeflection(j_young, momentExpr, daMoment)
    #at x = jointStartLocation, d(deflectionExpr1)/dx = d(deflectionExpr2), so the first constant is them subtracted from each other
    c1 = diff(deflectionExpr1, x).subs(x, jointStartLocation) - diff(draftdeflectionExpr2, x).subs(x, jointStartLocation)
    draftdeflectionExpr2 += c1 * x
    #at x = jointStartLocation, deflectionExpr1 = deflectionExpr2, so the second constant is them subtracted from each other
    c2 = deflectionExpr1.subs(x, jointStartLocation) - draftdeflectionExpr2.subs(x, jointStartLocation)
    deflectionExpr2 = draftdeflectionExpr2 + c2

    #calcuting deflection between (jointStartLocation + jointLength) and the end (wingLength)
    draftdeflectionExpr3 = calculateDeflection(young, momentExpr, daMoment)
    #at x = jointStartLocation + jointSize, d(deflectionExpr1)/dx = d(deflectionExpr2), so the third constant is them subtracted from each other
    c3 = diff(deflectionExpr2, x).subs(x, (jointStartLocation + jointSize)) - diff(draftdeflectionExpr3, x).subs(x, (jointStartLocation + jointSize))
    draftdeflectionExpr2 += c3 * x
    #at x = jointStartLocation + jointSize, deflectionExpr1 = deflectionExpr2, so the fourth constant is them subtracted from each other
    c4 = deflectionExpr2.subs(x, (jointStartLocation + jointSize)) - draftdeflectionExpr3.subs(x, (jointStartLocation + jointSize))
    deflectionExpr3 = draftdeflectionExpr3 + c4
    #print(deflectionExpr1)
    #print(deflectionExpr2)
    #print(deflectionExpr3)
    return [deflectionExpr1, deflectionExpr2, deflectionExpr3]


In [672]:
def getSecondMoment(wingRadius):
    return (np.pi/4 ) * (wingRadius ** 4)

In [673]:
def calculateDeflection(young, momentExpression, daSecondMoment):
    #integrate twice since EI * d^2(v)/dx^2 = M(x)     
    return (1/(young * daSecondMoment)) * integrate(integrate(momentExpression))

In [674]:
wingSim(wingWeight, young, j_young, air_load, wingLength, jointStartLocation, jointSize, wingRadius, angleOfWing, minimumStress, thresholdStress)

Minimum radius:  1.64703911685085
Maximum radius:  2.37544145855763
Minimum Young's Modulus:  4.31331719153434e-6
Maximum Young's Modulus:  1.86621129791041e-5
Bending stiffness (at the tip, EI(L)):  0.000107861473883809
Bending stiffness (at the wing base, EI(0)-->zero deflection):  11559406.3763944
Minimum wing length:  7.77366840828163e-7
Maximum wing length:  1.34643886443369e-6
